In [66]:
import os
import pandas as pd
import numpy as np
import json
from PIL import Image
from skimage import io, transform

# read the indexes of the left ears
f = open("./awe/lefts.txt", "r")
left_inds = []

for i in range(100):
    line = f.readline()
    x = [(int(s)- (10*i)) for s in line.split() if s.isdigit()]
    left_inds.append(x)
f.close()

In [67]:
# save an updated excel file:

file_name = "./awe-translation.csv"                #Path to your file
df = pd.read_csv(file_name) #Read Excel file as a DataFrame

subfolder_names = np.array( df['AWE image path'] )
isLeft_ids = np.zeros( subfolder_names.shape ).astype(np.int)
isMale = np.zeros( subfolder_names.shape ).astype(np.int)

# print( len(subfolder_names) )   # 1000   # 520 of those are left.

counter = 0
for i in range( len(subfolder_names) ):
    subfolder_name = (subfolder_names[i])[0:3]  # for some reason there are 4 channels.
    
    with open('./awe/' + subfolder_name +'/annotations.json') as f:
        data = json.load(f)
        gender = data["gender"]
        
    if (gender == 'm'):
        isMale[i] = 1
        counter += 1
        
    subfolder_int = int((subfolder_names[i])[0:3])
    image_id_1_10 = int((subfolder_names[i])[4:6])
    if ( image_id_1_10 in left_inds[subfolder_int-1] ):
        isLeft_ids[i] = 1
        # print("Left: ", subfolder_names[i])
        # counter += 1

print("Fraction of men:", counter, "/ 1000")
df['isLeft'] = isLeft_ids
df['isMale'] = isMale
# Display top 5 rows to check if everything looks good
# print(df.head(5))
#To save it back as Excel

# make new column:
pics_paths_2_npy = pics_paths_2.copy()
for i in range(len(pics_paths_2_npy)):
    pics_paths_2_npy[i] = (pics_paths_2_npy[i])[:-3] + 'npy'
# print(pics_paths_2_npy)

df['AWE-npy-path'] = pics_paths_2_npy
df.to_csv("./awe_isLeft.csv", index=False) #Write DateFrame back as csv file

Fraction of men: 910 / 1000


In [68]:
# put pictures in train and test folders and resize them:
file_name = "./awe-translation.csv"                #Path to your file
df = pd.read_csv(file_name) #Read csv file as a DataFrame

pics_paths   = np.array( df['AWE image path'] )
pics_paths_2 = np.array( df['AWE-Full image path'] )



# print( len(subfolder_names) )   # 1000   # 520 of those are left.
widths = []
heights = []
y_to_x_ratios=[]

for i in range( len(pics_paths) ):
    pic_path = pics_paths[i]
    pic_path_2 = pics_paths_2[i]
    pic_path_2_npy = (pics_paths_2[i])[:-3] + 'npy'
    img = np.array( io.imread('./awe/'+pic_path) )[:,:,0:3]  # for some reason the images are four dimensional.
    img_resized = transform.resize(img, (70, 40))
    # print(i, ":", img_resized.shape, type(img_resized))
    # print(i, ":", pic_path_2_npy)
    
    # print(pic_path, "   ", pic_path_2)
    # print(img.size[0], img.size[1])
    widths.append( img.shape[0] )
    heights.append( img.shape[1] )
    y_to_x_ratios.append( img.shape[1]/img.shape[0]  )
    
    # save the image in the newFolder:
    # print(img_resized[0:5,0:5,0])
    # print(np.max(np.max(img_resized[:,:,0])))
    # print(np.max(np.min(img_resized[:,:,0])))
    
    img_resized_normalized = img_resized.astype(np.float32).copy()
    img_resized_normalized = img_resized_normalized.swapaxes(0, 2)
    img_resized_normalized = img_resized_normalized.swapaxes(1, 2)
    
    img_resized = (img_resized*255).astype(np.uint8)
    
    # print(img_resized[0:5,0:5,0])
    # print(np.max(np.max(img_resized[:,:,0])))
    # print(np.max(np.min(img_resized[:,:,0])))
    
    
    # io.imshow(img_resized)
    
    # MOST IMPORTANT: SAVING AS THE CORRECT TYPE:
    io.imsave("./"+pic_path_2, img_resized)
    np.save("./"+pic_path_2_npy, img_resized_normalized)

print("widths:")
print("smallest:", min(widths))
print("highest:",  max(widths))
print("average:",  sum(widths)/len(widths))

print("\nheights:")
print("smallest:", min(heights))
print("highest:",  max(heights))
print("average:",  sum(heights)/len(heights))

print("\nratios:")
print("smallest:", min(y_to_x_ratios))
print("highest:",  max(y_to_x_ratios))
print("average:",  sum(y_to_x_ratios)/len(y_to_x_ratios))
# Display top 5 rows to check if everything looks good
# print(df.head(5))


widths:
smallest: 25
highest: 1022
average: 159.898

heights:
smallest: 12
highest: 489
average: 82.492

ratios:
smallest: 0.24285714285714285
highest: 1.1142857142857143
average: 0.5323459368277056


In [ ]:
# get the 